In [1]:
import scipy as sci
import joblib as job
import numpy as np
import pandas as pd
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import model_selection
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.linear_model import RidgeCV, LassoCV,ElasticNetCV, MultiTaskLasso, MultiTaskElasticNetCV
from sklearn.linear_model import MultiTaskLassoCV
from sklearn.metrics import r2_score
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
#import statsmodels.api as sm
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics as sm
from sklearn.preprocessing import MinMaxScaler

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
from sklearn.linear_model import RidgeCV, LassoCV,ElasticNetCV


In [2]:
import pyodbc

# Print available ODBC drivers
print(pyodbc.drivers())

# Connect to SQL Server
conn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};'
                      'SERVER="";'
                      'DATABASE=MeMawBestKnowBest;'
                      'user=cyfo123;'
                      'password=cyfo1234;'
                      'Trusted_Connection=yes;')
sql_query = "select  * from [MeMawBestKnowBest].[dbo].[Income_Rent_Data]"
conn.execute(sql_query)
inc_rent = pd.read_sql(sql_query, conn)
print(inc_rent)

['SQL Server', 'ODBC Driver 17 for SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'SQL Server Native Client 11.0', 'ODBC Driver 18 for SQL Server', 'Microsoft Access dBASE Driver (*.dbf, *.ndx, *.mdx)']


C:\Users\oladi\AppData\Roaming\Python\Python39\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


        NAME  pcincome  hrent     place          state  \
0      52800   32432.0  766.0      Olpe         Kansas   
1      52900    9028.0  955.0    Oneida         Kansas   
2      53200   26387.0  820.0     Osage         Kansas   
3      53525   21074.0  950.0      Otis         Kansas   
4      54125   33589.0  544.0     Palco         Kansas   
...      ...       ...    ...       ...            ...   
22799  63604   26810.0  510.0  Piedmont  West Virginia   
22800  63772   34114.0  864.0     Pinch  West Virginia   
22801  66412   12468.0  731.0  Quinwood  West Virginia   
22802  70828   15904.0  588.0    Rupert  West Virginia   
22803  75820   20127.0  580.0   Spencer  West Virginia   

                            Placeid  
0                 52800-Olpe-Kansas  
1               52900-Oneida-Kansas  
2                53200-Osage-Kansas  
3                 53525-Otis-Kansas  
4                54125-Palco-Kansas  
...                             ...  
22799  63604-Piedmont-West Virginia  

In [3]:
scaler = MinMaxScaler()
inc_rent[['income', 'rent']] = scaler.fit_transform(inc_rent[['pcincome', 'hrent']])
pd.set_option('display.max_columns', None)
print(inc_rent)

        NAME  pcincome  hrent     place          state  \
0      52800   32432.0  766.0      Olpe         Kansas   
1      52900    9028.0  955.0    Oneida         Kansas   
2      53200   26387.0  820.0     Osage         Kansas   
3      53525   21074.0  950.0      Otis         Kansas   
4      54125   33589.0  544.0     Palco         Kansas   
...      ...       ...    ...       ...            ...   
22799  63604   26810.0  510.0  Piedmont  West Virginia   
22800  63772   34114.0  864.0     Pinch  West Virginia   
22801  66412   12468.0  731.0  Quinwood  West Virginia   
22802  70828   15904.0  588.0    Rupert  West Virginia   
22803  75820   20127.0  580.0   Spencer  West Virginia   

                            Placeid    income      rent  
0                 52800-Olpe-Kansas  0.086256  0.196061  
1               52900-Oneida-Kansas  0.016884  0.251617  
2                53200-Osage-Kansas  0.068338  0.211934  
3                 53525-Otis-Kansas  0.052590  0.250147  
4            

In [6]:

# City Score: Make rend negative and create a score for each city
int_rent_place=inc_rent[['place', 'state', 'Placeid']] 
inc_rent['rent'] *= -1
inc_rent_data_v1 = inc_rent[['income', 'rent']]
# Generate random numbers for all iterations and Initialize an empty DataFrame to store CityScores for each iteration and Calculate CityScore 
# with random numbers added for each iteration and  Calculate the average CityScore for each row across all iterations
iterations = 10
ran_num = np.random.rand(iterations, len(inc_rent_data_v1))
CityScore_df = pd.DataFrame(index=inc_rent_data_v1.index, columns=range(iterations))
for i in range(iterations):
    CityScore_df[i] = 0.5 * inc_rent_data_v1['income'] + 0.5 * inc_rent_data_v1['rent'] + ran_num[i]
CityScore = CityScore_df.mean(axis=1)

inc_rent_data = pd.DataFrame({'CityScore': CityScore, 'income': inc_rent_data_v1['income'], 'rent': inc_rent_data_v1['rent']})

y_inc_rent_data=inc_rent_data['CityScore']
x_inc_rent_data=inc_rent_data.drop(['CityScore'],axis=1)

print(CityScore)

0        0.807290
1        0.774203
2        0.545462
3        0.757908
4        0.582970
           ...   
22799    0.638033
22800    0.752396
22801    0.665225
22802    0.401839
22803    0.714750
Length: 22804, dtype: float64


In [7]:
#Create Test Size
Testsize=round(0.0*len(inc_rent_data.index))
y_inc_rent_score_train=y_inc_rent_data[:-Testsize]
y_inc_rent_score_test=y_inc_rent_data[-Testsize:]
x_inc_rent_score_train=x_inc_rent_data[:-Testsize]
x_inc_rent_score_test=x_inc_rent_data[-Testsize:]





#Store Data
%store y_inc_rent_score_train
%store y_inc_rent_score_test
%store x_inc_rent_score_train
%store x_inc_rent_score_test
%store y_inc_rent_data
%store x_inc_rent_data
%store int_rent_place


Stored 'y_inc_rent_score_train' (Series)
Stored 'y_inc_rent_score_test' (Series)
Stored 'x_inc_rent_score_train' (DataFrame)
Stored 'x_inc_rent_score_test' (DataFrame)
Stored 'y_inc_rent_data' (Series)
Stored 'x_inc_rent_data' (DataFrame)
Stored 'int_rent_place' (DataFrame)
